<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import sys
import glob
import json
import datetime as dt
from pathlib import Path
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from dotenv import load_dotenv
import pandas_ta as ta

pd.set_option("display.max_columns", 40)
pd.set_option('display.max_rows', 500)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\User\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Carregar variáveis do arquivo .env
load_dotenv()


True

In [3]:
def get_mt5_credentials():
    """Carrega credenciais do MT5 do arquivo .env"""
    return {
        'login': int(os.getenv('MT5_LOGIN', 0)),
        'password': os.getenv('MT5_PASSWORD', ''),
        'server': os.getenv('MT5_SERVER', ''),
        'path': os.getenv('MT5_PATH', r"C:\Program Files\MetaTrader 5\terminal64.exe")
    }


def load_config(config_file):
    """Carrega todas as configurações do arquivo JSON"""
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)
        return config
    except FileNotFoundError:
        print(f"Arquivo {config_file} não encontrado!")
        return None
    except json.JSONDecodeError:
        print(f"Erro ao decodificar JSON do arquivo {config_file}")
        return None
    except Exception as e:
        print(f"Erro ao carregar {config_file}: {e}")
        return None


def connect_mt5(config=None):
    """Estabelece conexão com MT5"""
    if config is None:
        config = get_mt5_credentials()
    
    # Validar credenciais
    if not config['login'] or not config['password'] or not config['server']:
        print("Erro: Credenciais do MT5 não encontradas no arquivo .env")
        print("Certifique-se de que MT5_LOGIN, MT5_PASSWORD e MT5_SERVER estão definidos")
        return False
    
    if not mt5.initialize(
        login=config['login'], 
        server=config['server'], 
        password=config['password'], 
        path=config['path']
    ):
        print("initialize() failed, error code =", mt5.last_error())
        return False
    
    print('Ligado ao MT5 com sucesso!')
    print(f'Conta: {config["login"]} | Servidor: {config["server"]}')
    print('-' * 20)
    return True

In [4]:
connect_mt5()

Ligado ao MT5 com sucesso!
Conta: 3008705129 | Servidor: Rico-DEMO
--------------------


True

In [5]:
def get_timeframe_offset(timeframe):
    """Retorna o offset em minutos baseado no timeframe"""
    timeframe_map = {
        't1': 1, 't2': 2, 't5': 5, 't10': 10, 't15': 15, 't30': 30,
        'h1': 60, 'h4': 240, 'd1': 1440,
    }
    return timeframe_map.get(timeframe.lower(), 5)


def trade_report(symbol, data_ini, data_fim, cost_per_lot=0.25):
    """Extrai relatório de trades do MT5"""
    deals = mt5.history_deals_get(data_ini, data_fim, group=symbol)
    
    if deals is None or len(deals) == 0:
        print(f"Nenhum deal encontrado para {symbol} no período especificado")
        return pd.DataFrame()
    
    print(f"Encontrados {len(deals)} deals para {symbol}")
    
    df = pd.DataFrame(list(deals), columns=deals[0]._asdict().keys())
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['custo'] = df['volume'] * cost_per_lot
    df['lucro'] = df['profit'] - df['custo']
    
    return df




In [6]:
data_ini = '2025-06-25'
data_fim = '2025-06-29'
symbol = '*WDO*'
cost_per_lot = 0.5

# Extrair dados do MT5
dfmt5 = trade_report(
    symbol, 
    pd.Timestamp(data_ini), 
    pd.Timestamp(data_fim ) + dt.timedelta(days=1),
    cost_per_lot
)

Encontrados 66 deals para *WDO*


In [12]:
dfmt5[~dfmt5['comment'].str.contains('\[')]

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id,custo,lucro
0,1572116679,2129866498,2025-06-25 10:10:00,1750846200692,0,0,1212,2129866498,3,1.0,5534.0,0.0,0.0,0.0,0.0,WDON25,gold_rsi_trend,,0.5,-0.5
1,1572377108,2130234928,2025-06-25 11:05:00,1750849500436,0,0,1212,2130234928,3,1.0,5549.0,0.0,0.0,0.0,0.0,WDON25,gold_rsi_trend,,0.5,-0.5
3,1572464971,2130359180,2025-06-25 11:30:01,1750851001112,0,0,1212,2130359180,3,1.0,5554.0,0.0,0.0,0.0,0.0,WDON25,gold_rsi_trend,,0.5,-0.5
4,1572556225,2130512873,2025-06-25 12:05:00,1750853100579,1,0,3443,2130512873,3,1.0,5559.0,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5
5,1572605045,2130587790,2025-06-25 12:20:01,1750854001023,1,0,3443,2130587790,3,1.0,5564.5,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5
10,1572677088,2130696417,2025-06-25 12:40:01,1750855201046,0,0,3443,2130696417,3,1.0,5560.0,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5
11,1572745112,2130785788,2025-06-25 13:00:00,1750856400673,1,0,3443,2130785788,3,1.0,5563.5,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5
13,1572760243,2130805579,2025-06-25 13:05:00,1750856700608,0,0,3443,2130805579,3,1.0,5555.5,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5
15,1572803368,2130866836,2025-06-25 13:30:00,1750858200925,0,0,3443,2130866836,3,1.0,5545.5,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5
18,1573019250,2131148652,2025-06-25 15:25:00,1750865100557,0,0,3443,2131148652,3,1.0,5552.0,0.0,0.0,0.0,0.0,WDON25,bb_anti_trend,,0.5,-0.5


In [13]:
magic_number = 1212
timeframe = '5'

# Dataframe com entradas (trades com magic_number correto)
filtro_ent = (dfmt5['magic']==magic_number) & (~dfmt5['comment'].str.contains('\['))
dfmt5_ent = dfmt5[filtro_ent][['time', 'type', 'position_id', 'magic', 'price', 'volume']].copy()


dfmt5_ent.rename(columns={'price': 'price_ent', 'time': 'time_ent'}, inplace=True)
dfmt5_ent.loc[dfmt5_ent['type'] == 1, 'posi'] = 'short'
dfmt5_ent.loc[dfmt5_ent['type'] == 0, 'posi'] = 'long'

# Dataframe com saídas
filtro_ext = (dfmt5['magic']==0) | ((dfmt5['magic']==magic_number) & (dfmt5['comment'].str.contains('\[')))
dfmt5_ext = dfmt5[filtro_ext][['time', 'position_id', 'price', 'profit', 'comment']].copy()


dfmt5_ext.rename(columns={'price': 'price_ext', 'time': 'time_ext'}, inplace=True)

# Unindo entradas e saídas
dfmt5_2 = pd.merge(dfmt5_ent, dfmt5_ext, on="position_id", how='left')


# Calculando métricas
dfmt5_2['delta_t'] = (dfmt5_2['time_ext'] - dfmt5_2['time_ent']).dt.total_seconds() / 60
dfmt5_2['pts_final_demo'] = abs(dfmt5_2['price_ext'] - dfmt5_2['price_ent'])

# Pontos finais considerando direção
dfmt5_2.loc[dfmt5_2['posi'] == 'long', 'pts_final_real'] = dfmt5_2['price_ext'] - dfmt5_2['price_ent']
dfmt5_2.loc[dfmt5_2['posi'] == 'short', 'pts_final_real'] = dfmt5_2['price_ent'] - dfmt5_2['price_ext']

# Ajustando tempo para comparação com candles baseado no timeframe
timeframe_offset = get_timeframe_offset(timeframe)
dfmt5_2['time'] = dfmt5_2['time_ent'] - pd.Timedelta(minutes=timeframe_offset)
dfmt5_2['time'] = dfmt5_2['time'].dt.round('min')
dfmt5_2.set_index("time", inplace=True)

# Lucro com custo
dfmt5_2['lucro'] = dfmt5_2['profit'] - dfmt5_2['volume'] * cost_per_lot

# Filtrando por magic number
dfmt5_2 = dfmt5_2[dfmt5_2['magic'] == magic_number].copy()


# Lucro acumulado
dfmt5_2['cstrategy'] = dfmt5_2['lucro'].cumsum()

In [14]:
dfmt5_2

,time_ent,type,position_id,magic,price_ent,volume,posi,time_ext,price_ext,profit,comment,delta_t,pts_final_demo,pts_final_real,lucro,cstrategy
time,,,,,,,,,,,,,,,,
2025-06-25 10:05:00,2025-06-25 10:10:00,0,2129866498,1212,5534.0,1.0,long,2025-06-25 11:28:19,5554.0,200.0,[tp 5554.000],78.316667,20.0,20.0,199.5,199.5
2025-06-25 11:00:00,2025-06-25 11:05:00,0,2130234928,1212,5549.0,1.0,long,2025-06-25 12:23:37,5569.0,200.0,[tp 5569.000],78.616667,20.0,20.0,199.5,399.0
2025-06-25 11:25:00,2025-06-25 11:30:01,0,2130359180,1212,5554.0,1.0,long,2025-06-25 12:26:05,5574.0,200.0,[tp 5574.000],56.066667,20.0,20.0,199.5,598.5
2025-06-26 10:40:00,2025-06-26 10:45:00,1,2131910396,1212,5519.5,1.0,short,2025-06-26 13:55:32,5499.5,200.0,[tp 5499.500],190.533333,20.0,20.0,199.5,798.0
2025-06-26 10:50:00,2025-06-26 10:55:00,1,2131960039,1212,5516.5,1.0,short,2025-06-26 14:41:54,5496.5,200.0,[tp 5496.500],226.900000,20.0,20.0,199.5,997.5
2025-06-26 11:40:00,2025-06-26 11:45:00,1,2132157873,1212,5510.5,1.0,short,2025-06-26 17:54:00,5493.5,170.0,,369.000000,17.0,17.0,169.5,1167.0
2025-06-26 14:00:00,2025-06-26 14:05:00,1,2132488646,1212,5501.0,1.0,short,2025-06-26 17:43:17,5493.0,80.0,[tp 5493.000],218.283333,8.0,8.0,79.5,1246.5
2025-06-26 14:30:00,2025-06-26 14:35:01,1,2132553549,1212,5500.0,1.0,short,2025-06-26 17:43:17,5492.0,80.0,[tp 5492.000],188.266667,8.0,8.0,79.5,1326.0
2025-06-26 14:45:00,2025-06-26 14:50:00,1,2132587856,1212,5497.5,1.0,short,2025-06-26 17:54:00,5493.5,40.0,,184.000000,4.0,4.0,39.5,1365.5


In [67]:
df=pd.read_csv('backtest_results/backtest_WIN@N_t5_pattern_rsi_trend_magic_1111.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')

In [68]:
df[df['position']!=0].index

DatetimeIndex(['2025-06-25 10:40:00', '2025-06-25 12:20:00',
               '2025-06-25 17:20:00', '2025-06-25 17:25:00',
               '2025-06-27 10:00:00'],
              dtype='datetime64[ns]', name='time', freq=None)

In [75]:
df.to_excel('check.xlsx')

In [85]:
??ta.rsi

In [81]:
df['rsi'] = df.ta.rsi(length=9)
df['pct_change'] = df['close'].pct_change().fillna(0)
long_condition = (df['pct_change'] > 0) & (df['rsi'] > 74)
short_condition = (df['pct_change'] < 0) & (df['rsi'] < 26)

df['position_2'] = np.where(long_condition, 1, np.where(short_condition, -1, 0))

In [84]:
df[df['position_2']!=0].index

DatetimeIndex(['2025-06-25 10:40:00', '2025-06-25 12:20:00',
               '2025-06-25 15:05:00', '2025-06-25 17:20:00',
               '2025-06-25 17:25:00', '2025-06-26 09:00:00',
               '2025-06-26 09:05:00', '2025-06-26 09:50:00',
               '2025-06-27 09:20:00', '2025-06-27 09:25:00',
               '2025-06-27 09:30:00', '2025-06-27 09:45:00',
               '2025-06-27 09:50:00', '2025-06-27 09:55:00',
               '2025-06-27 10:00:00', '2025-06-27 14:50:00'],
              dtype='datetime64[ns]', name='time', freq=None)

In [90]:
df[['close', 'pct_change', 'rsi','position','position_2']]

,close,pct_change,rsi,position,position_2
time,,,,,
2025-06-25 09:00:00,139055.0,0.000000,NaN,0,0
2025-06-25 09:05:00,139120.0,0.000467,NaN,0,0
2025-06-25 09:10:00,138865.0,-0.001833,NaN,0,0
2025-06-25 09:15:00,138530.0,-0.002412,NaN,0,0
2025-06-25 09:20:00,138495.0,-0.000253,NaN,0,0
2025-06-25 09:25:00,138380.0,-0.000830,NaN,0,0
2025-06-25 09:30:00,138560.0,0.001301,NaN,0,0
2025-06-25 09:35:00,138565.0,0.000036,NaN,0,0
2025-06-25 09:40:00,138500.0,-0.000469,NaN,0,0


In [82]:
df.to_excel('check.xlsx')

In [69]:
dfmt5_2.index

DatetimeIndex(['2025-06-25 10:00:00', '2025-06-25 10:15:00',
               '2025-06-25 10:25:00', '2025-06-25 10:30:00',
               '2025-06-25 10:35:00', '2025-06-25 10:40:00',
               '2025-06-25 12:10:00', '2025-06-25 12:20:00',
               '2025-06-25 17:20:00', '2025-06-25 17:25:00',
               '2025-06-26 10:15:00', '2025-06-27 10:00:00'],
              dtype='datetime64[ns]', name='time', freq=None)